# **CODE on Colab**

`Welcome to a new notebook. As I showed you a previous notebook called Colab RDP, This notebook feature VSCode (If possible going to add more) integration on web browser using colab.`

`This notebook going feature a 12 hour (max) of runtime of VS Code on a website with colab power`

In [ ]:
# @title VSCode

# Imports
import os
import requests
import re
import shlex
import time
import signal

from subprocess import Popen, PIPE, call

# @markdown This code cell runs [code-server](https://github.com/cdr/code-server) with agro tunnel.

# # @markdown Mount GDrive
Gdrive = False  # param {type: 'boolean'}

# @markdown Password (Can be None)
Password = ""  # @param {type: 'string'}

# @markdown Sleep
Sleep = True  # @param {type: 'boolean'}

# @markdown Run this cell to start vscode

class ColabCode:
    def __init__(
            self,
            port=10000,
            password=None,
            mount_drive=False,
            verbose=False,
            sleep=False):

        self.install_server(verbose)
        self.start_code(port=port, password=password, verbose=verbose)

        if sleep:
            time.sleep(43200)

    @staticmethod
    def __get_agro():
        hostname = None
        output = requests.get("http://localhost:45678/metrics").text
        result = re.search('cloudflared_tunnel_user_hostnames_counts{userHostname="https://(.+?)"}', output)
        if result is None:
            raise Exception("Cannot get any result from cloudflared metrics")
        hostname = result.group(1)
        if hostname is None:
            raise Exception("Cannot get the hostname from cloudflared, it looks like the connection has failed.")
        return {
            "domain": hostname,
            "protocol": "",
            "port": 22
        }

    def install_server(self, verbose=False):
        os.system("kill -9 $(ps aux | grep 'cloudflared' | awk '{print $2}')")

        if not os.path.isfile("cloudflared"):
            ! wget - q - nc https: // bin.equinox.io/c/VdrWdbjqyF/cloudflared-stable-linux-amd64.tgz
            ! tar zxf cloudflared-stable-linux-amd64.tgz
            ! curl - fsSL https: // code-server.dev/install.sh | sh
        else:
            if verbose:
                print("DEBUG: Skipping installation")

    def start_code(self, port=10000, password=None, verbose=False):
        if password:
            code_cmd = f'PASSWORD={password} code-server --port {port} --disable-telemetry'
        else:
            code_cmd = f'code-server --port {port} --auth none --disable-telemetry'

        agro_cmd = f'./cloudflared tunnel --url http://127.0.0.1:10000 --metrics localhost:45678'

        # ! PASSWORD=$password code-server --port $port --disable-telemetry &
        code_proc = Popen([code_cmd], shell=True, stdout=PIPE)
        agro_proc = Popen(shlex.split(agro_cmd), stdout=PIPE)

        if verbose:
            print(f"Code Server process: {code_proc.pid}")
            print(f"Cloudflared process: {agro_proc.pid}")

        time.sleep(5)
        try:
            info = self.__get_agro()
            print(f'VSCode Server is ready, Visit https://{info["domain"]}')
        except:
            # os.kill(code_proc.pid, signal.SIGKILL)
            os.kill(agro_proc.pid, signal.SIGKILL)
            print(f"FAILED: Killing Server... Retry again ...")


if __name__ == "__main__":
    if Password is not "":
        ColabCode(password=Password, mount_drive=Gdrive, verbose=True, sleep=Sleep)
    else:
        ColabCode(mount_drive=Gdrive, verbose=True, sleep=Sleep)


# **END NOTE**

`THANK YOU FOR YOUR ALL LOVE AND SUPPORT, GOING TO ADD MORE FUNCTIONALITY SOON`
